# Template Matching Configurations

This example notebook outlines the steps necessary to generate, save, and load configurations for the `match-template` program through Python object and `yaml` files.
Here, we focus on *how* to create and modify these configurations rather than the underlying code for parsing these configurations and running the program.

### Rationale for using YAML configurations

While the `Py2DTM` package provides a basic CLI program and an object-oriented Python API for extending template matching into more complex workflows, it is useful to have a human-readable, easily editable, and shareable configuration file because:
1. It increases reproducibility by keeping a record of exact parameters used for a particular run,
2. It can be quickly modified during development, debugging, and testing without changing underlying code, and
3. It can be replicated across large datasets (e.g. multiple images with similar configurations) for execution on distributed clusters.

We find that storing configurations in a structured file format strikes a good balance between user-friendliness and programmatic control.

## Importing Necessary Classes and Functions

We utilize [Pydantic](https://docs.pydantic.dev/latest/) to create Python objects that parse, validate, and serialize configurations.
These objects (called Pydantic models) are laid out in a hierarchial structure with a single root "manager" model.
Below we import all the configuration classes (along with other libraries) we will detail usage of in this notebook.

In [1]:
from pprint import pprint

from tt2dtm.pydantic_models import (
    OpticsGroup,
)

## The **OpticsGroup** Model

The `OpticsGroup` model is a container for microscope imaging parameters necessary for calculating filters (e.g. contrast transfer functions).
We follow the fields that are defined in [RELION's](https://relion.readthedocs.io/en/latest/) optics group .star file, and the class has the following attributes:
- `label`: A unique label for the optics group, usually contains some form of the micrograph name but can be any string.
- `pixel_size`: Float value representing the pixel size of the image, in Angstroms.
- `voltage`: The voltage of the microscope, in kV.
- `spherical_aberration`: The spherical aberration of the microscope, in mm, with the default value of 2.7 mm.
- `amplitude_contrast_ratio`: The amplitude contrast ratio (unitless) with the default value of 0.07.
- `phase_shift`: Additional phase shift to apply across the CTF, in degrees, with the default value of 0.0.
- `defocus_u`: Defocus of the micrograph along the major axis, in Angstroms.
- `defocus_v`: Defocus of the micrograph along the minor axis, in Angstroms.
- `defocus_astigmatism_angle`: Angle of the defocus astigmatism (relative to the x-axis), in degrees. The default value is 0.0.
- `ctf_B_factor`: An additional b-factor to apply to the CTF, in Angstroms^2. The default value is 0.0.

There are other unused fields in the class that are not detailed here.
See the Pydantic model API documentation for more information.

### Creating an instance of the **OpticsGroup** model

Below, we create an instance of the `OpticsGroup` model with some made-up but nevertheless realistic values.

In [2]:
my_optics_group = OpticsGroup(
    label="my_optics_group",
    pixel_size=1.06,
    voltage=300.0,
    spherical_aberration=2.7,  # default value
    amplitude_contrast_ratio=0.07,  # default value
    phase_shift=0.0,  # default value
    defocus_u=5200.0,
    defocus_v=4950.0,
    defocus_astigmatism_angle=25.0,
    ctf_B_factor=60.0,
)

The Python variable `my_optics_group` is now an instance of the `OpticsGroup` model.
Note that the model does do validation under-the-hood to ensure necessary fields are present and valid.
Any invalid fields will raise a `ValidationError` when the model is created.
Uncomment the following code block to see this in action.

In [10]:
# bad_optics_group = OpticsGroup(
#     label="bad_optics_group",
#     pixel_size=-1.0,  # <--- Must be positive
#     voltage=300.0,
#     phase_shift=0.0,  # default value
#     defocus_u=5200.0,
#     defocus_v=4950.0,
#     defocus_astigmatism_angle=25.0,
# )

### Serializing an instance of the **OpticsGroup** model

Pydantic has built-in functionality, namely the `model_dump()`, for generating a dictionary of key, value pairs from the model attributes and their values.
Below, we create a dictionary from the `my_optics_group` instance and print it out.
Note that extra, unused fields are still included in the dictionary.

In [4]:
optics_dict = my_optics_group.model_dump()
pprint(optics_dict)

{'amplitude_contrast_ratio': 0.07,
 'beam_tilt_x': None,
 'beam_tilt_y': None,
 'chromatic_aberration': 0.0,
 'ctf_B_factor': 60.0,
 'defocus_astigmatism_angle': 25.0,
 'defocus_u': 5200.0,
 'defocus_v': 4950.0,
 'even_zernike': None,
 'label': 'my_optics_group',
 'mtf_reference': None,
 'mtf_values': None,
 'odd_zernike': None,
 'phase_shift': 0.0,
 'pixel_size': 1.06,
 'spherical_aberration': 2.7,
 'voltage': 300.0,
 'zernike_moments': None}


### Exporting configurations to a YAML file

[YAML](https://yaml.org) files are nothing more than a bunch of key-value pairs in a human-readable format.
Like [JSON](https://www.json.org), YAML has parser functions/libraries in most programming languages increasing their interoperability.
We adopt the `.yaml` format (and `.json` format, but not discussed here) for our configuration files rather than less-common formats specific to a sub-field or program.

The `OpticsGroup` model (and all the other Pydanic models discussed here) have a `to_yaml()` method that writes the model to a YAML file.
Below, we first specify a path and then call the `to_yaml()` method on the `my_optics_group` instance to write the model to a file.

In [6]:
yaml_filepath = "./optics_group_example.yaml"
my_optics_group.to_yaml(yaml_filepath)

A new file called `optics_group_example.yaml` should now exist in the current working directory with the following contents:

----

```yaml
amplitude_contrast_ratio: 0.07
beam_tilt_x: null
beam_tilt_y: null
chromatic_aberration: 0.0
ctf_B_factor: 60.0
defocus_astigmatism_angle: 25.0
defocus_u: 5200.0
defocus_v: 4950.0
even_zernike: null
label: my_optics_group
mtf_reference: null
mtf_values: null
odd_zernike: null
phase_shift: 0.0
pixel_size: 1.06
spherical_aberration: 2.7
voltage: 300.0
zernike_moments: null
```

----

### Importing configurations from a YAML file

Each model also has the `from_yaml()` method which can be to instantiate the class from contents in a `.yaml` file.
Below, we are creating a new instance of the `OpticsGroup` class from the `optics_group.yaml` file.

In [ ]:
new_optics_group = OpticsGroup.from_yaml(yaml_filepath)

# Check if the two OpticsGroup objects are equal
if new_optics_group == my_optics_group:
    print("OpticsGroup objects are equal.")
else:
    print("The two OpticsGroup are not equal!!!")

OpticsGroup objects are equal.


Now that we've covered the basics of creating, serializing, and deserializing the `OpticsGroup` model, we can move onto the next models without covering the (de)serialization and import/export steps in detail.

## The **OrientationSearchConfig** Model

Two-dimensional template matching necessitates covering SO(3) orientation space to find the "best" orientation match for a particle.
How points are sampled during the search process is handled by the `OrientationSearchConfig` model.

This model effectively acts as an interface with the [torch-so3](https://github.com/teamtomo/torch-so3) package, which provides the underlying functionality for generating uniform grids on SO(3).
The class has the following attributes:
TODO

## The **DefocusSearchConfig** Model

Two-dimensional template matching is also sensitive to the relative defocus of a particle allowing the estimation of the Z-height in a sample.
The `DefocusSearchConfig` model handles which defocus planes are searched over (relative to the defocus parameters defined in the `OpticsGroup` model).
The model has the following attributes:
TODO

The **BandpassFilterConfig** Model


The **PhaseRandomizationFilterConfig** Model


The **PreprocessingFilters** Model


The **WhiteningFilterConfig** Model


The **MatchTemplateManager** Model


The **MatchTemplateResult** Model
